# Sinteni blokovi

In [1]:
class GenomeGraph:
    '''Inicijalizacija grafa pomoću neusmerenih grana'''
    def __init__(self, edges):
        self.adjacency_list = {}
        
        for (u, v) in edges:
            if u not in self.adjacency_list:
                self.adjacency_list[u] = []
                
            if v not in self.adjacency_list:
                self.adjacency_list[v] = []
                
            self.adjacency_list[u].append(v)
            self.adjacency_list[v].append(u)
            
        
    '''Metod za pronalaženje ciklusa u grafu'''
    def get_cycles(self):
        unvisited = set(self.adjacency_list.keys())
        cycles = []
        
        while len(unvisited) > 0:
            v = min(unvisited)
            unvisited.remove(v)
            current_cycle = [v]
            
            while True:
                new_v = None
                for w in sorted(self.adjacency_list[v]):
                    if w in unvisited:
                        unvisited.remove(w)
                        new_v = w
                        break
                        
                if new_v == None:
                    break
                    
                current_cycle.append(new_v)
                v = new_v

                
            cycles.append(current_cycle)
            
        return cycles
    
    '''Dodavanje neusmerene grane u graf'''
    def add_edge(self, i, j):
        self.adjacency_list[i].append(j)
        self.adjacency_list[j].append(i)
        
    '''Uklanjanje neusmerene grane u graf'''
    def remove_edge(self, i, j):
        self.adjacency_list[i].remove(j)
        self.adjacency_list[j].remove(i)
            
    '''Modifikacija grafa uvodjnjem 2-break transformacije'''
    def two_break_on_genome_graph(self, i, i_p, j, j_p):
        self.remove_edge(i, i_p)
        self.remove_edge(j, j_p)
        
        self.add_edge(i, j)
        self.add_edge(i_p, j_p)

        return self
    
    '''Formatiranje ispisa'''
    def __str__(self):
        return f'{self.adjacency_list}'

In [2]:
class Reversals:
    def k_sorting_reversal(self, P, k):
        n = len(P)
        for i in range(k, n):
            if abs(P[i]) == k + 1:
                P[k:i+1] = [-x for x in P[:][k:i+1][::-1]] 
                return P

    '''Pohlepni algoritam za pronalaženje najmanjeg broja transformacija hromozoma'''
    def greedy_sorting(self, P, verbose=False):
        approx_reversal_distance = 0
        
        if verbose:
            print(P)
        
        n = len(P)
        for k in range(n):
            if P[k] != (k + 1):
                P = self.k_sorting_reversal(P, k)
                approx_reversal_distance += 1
                
                if verbose:
                    print(P)
                
                if P[k] == -(k + 1):
                    P[k] = -P[k]
                    approx_reversal_distance += 1
                    
                    if verbose:
                        print(P)
        return approx_reversal_distance
    
    '''Transformacija hromozoma u niz čvorova ciklusa'''
    def chromosome_to_cycle(self, chromosome):
        n = len(chromosome)
        nodes = [0] * (2 * n)
        for j in range(n):
            i = chromosome[j]
            
            if i > 0:
                nodes[2 * j] = (2 * i) - 1
                nodes[2 * j + 1] = 2 * i
                
            else:
                nodes[2 * j] = -2 * i
                nodes[2 * j  + 1] = -(2 * i) - 1
                
        return nodes
                    
        
    '''Transformacija niza čvorova ciklusa u hromozom'''
    def cycle_to_chromosome(self, nodes):
        m = len(nodes)
        chromosome = [0] * (m // 2)
        for j in range(0, m - 1, 2):
            if nodes[j] < nodes[j + 1]:
                chromosome[j // 2] = nodes[j + 1] // 2
            else:
                chromosome[j // 2] = - nodes[j] // 2
        return chromosome
    
    '''Izdvajanje obojenih grana ciklusa hromozoma'''
    def colored_edges(self, P):
        edges = []
        for chromosome in P:
            nodes = self.chromosome_to_cycle(chromosome)
            
            m = len(nodes)
            for j in range(1, m - 1, 2):
                edges.append((nodes[j], nodes[j + 1]))
                
            edges.append((nodes[-1], nodes[0]))
        return edges
    
    '''Izdvajanje crnih grana ciklusa hromozoma'''
    def black_edges(self, P):
        edges = []
        for chromosome in P:
            nodes = self.chromosome_to_cycle(chromosome)
            
            m = len(nodes)
            for j in range(0, m - 1, 2):
                edges.append((nodes[j], nodes[j + 1]))
                
        return edges
    
    '''Transformacija grafa u genom'''
    def graph_to_genome(self, genome_graph):
        P = []
        cycles = genome_graph.get_cycles()
        
        for nodes in cycles:
            chromosome = self.cycle_to_chromosome(nodes)
            P.append(chromosome)
            
        return P
    
    
    '''Uvođenje 2-break transformacije u genom'''
    def two_break_on_genome(self, P, i, i_p, j, j_p):
        genome_graph = GenomeGraph(self.black_edges(P) + self.colored_edges(P))
        genome_graph.two_break_on_genome_graph(i, i_p, j, j_p)
        return self.graph_to_genome(genome_graph)
    
    '''Pronalaženje najmanjeg broja transformacija jednog genoma u drugi'''
    def shortest_rearrangement_scenario(self, P, Q):
        print(P, '\n')
        red_edges = self.colored_edges(P)
        blue_edges = self.colored_edges(Q)
        breakpoint_graph = GenomeGraph(red_edges + blue_edges)
        
        while True:
            cycles = breakpoint_graph.get_cycles()
            non_trivial_cycle = None
            
            for cycle in cycles:
                if len(cycle) > 2:
                    non_trivial_cycle = cycle
                    break
                    
            if non_trivial_cycle != None:
                n = len(non_trivial_cycle)
                cycle_edges = [
                    (non_trivial_cycle[i], non_trivial_cycle[i + 1]) for i in range(n - 1)
                ] + [(non_trivial_cycle[-1], non_trivial_cycle[0])]

            
            else:
                break 
            
            for k in range(n):
                edge = cycle_edges[k]
                (j_blue, i_p_blue) = edge
                if (j_blue, i_p_blue) in blue_edges or (i_p_blue, j_blue) in blue_edges:
                    previous_red_edge = cycle_edges[(k - 1) % n]
                    next_red_edge = cycle_edges[(k + 1) % n]
                    break

            # Ispisi korisni tokom testiranja / debagovanja
            # print(f'Blue edge: {(j_blue, i_p_blue)}')
            # print(f'Previus red edge: {previous_red_edge}')
            # print(f'Next red edge: {next_red_edge}')
            
            (i, j) = previous_red_edge            
            (i_p, j_p) = next_red_edge
            
            breakpoint_graph.remove_edge(i, j)
            breakpoint_graph.remove_edge(i_p, j_p)
            
            breakpoint_graph.add_edge(i_p, j)
            breakpoint_graph.add_edge(i, j_p)
            
            P = self.two_break_on_genome(P, j, i, i_p, j_p)
            print(P)
            print() # Ispis praznog reda radi boljeg raspoznavanja iteracija
        print(P)

## Testovi

In [3]:
# Ukloniti komentare radi pokretanja pojedinačnih testova
# -------------------------------------------------------
revs = Reversals()

P = [[+1, -7, +6, -10, +9, -8, +2, -11, -3, +5, +4]]
Q = [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]

# P = [-6, +1, +2, +3, +4, +5]

# P = [[1, -2, -3, 4]]
# Q = [[1, 2, 3, 4]]

# P = [[-6, +1, +2, +3, +4, +5]]
# Q = [[1, 2, 3, 4, 5, 6]]

# chromosome = [1, -2, -3, 4]
# P = [chromosome]

# revs.greedy_sorting(P, verbose=True)

# revs.cycle_to_chromosome(revs.chromosome_to_cycle(chromosome))
# print(revs.colored_edges(P))
# print(revs.black_edges(P))
# edges = revs.colored_edges(P) + revs.black_edges(P)
# genome_graph = GenomeGraph(edges)

# print(genome_graph.adjacency_list)

# revs.graph_to_genome(genome_graph)
# genome_graph.two_break_on_genome_graph(1, 8, 3, 6)
# genome_graph.get_cycles()

# revs.two_break_on_genome(P, 1, 8, 3, 6)
revs.shortest_rearrangement_scenario(P, Q)

[[1, -7, 6, -10, 9, -8, 2, -11, -3, 5, 4]] 

[[1, -7, 6, -10, 9, -8, 2, -11, -3], [4, 5]]

[[1, -7, 6, -10, 9, -8, 2, 3, 11], [4, 5]]

[[1, 2, 3, 11], [4, 5], [6, -10, 9, -8, -7]]

[[1, 2, 3, 4, 5, 11], [6, -10, 9, -8, -7]]

[[1, 2, 3, 4, 5, 6, -10, 9, -8, -7, 11]]

[[1, 2, 3, 4, 5, 6, 7, 8, -9, 10, 11]]

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]
